In [ ]:
from probinf.distributions import *
from probinf.bayes_net import *
from probinf.inference import *
from probinf.utils import *
from probinf.xl_reader import *

# 4.1	Installation

In [ ]:
!pip install -r requirements.txt

# 4.2	Netze definieren

In [ ]:
einbruch_netz = BayesNet([
    ('Einbruch', '', 0.001),
    ('Erdbeben', '', 0.002),
    ('Alarm', 'Einbruch Erdbeben',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnRuftAn', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryRuftAn', 'Alarm', {T: 0.70, F: 0.01}) ])

grippe_netz = BayesNet([
    ('Geimpft', [], 0.60),
    ('Grippe', ['Geimpft'], {T: 0.002, F: 0.02}),
    ('Fieber', ['Grippe'], {T: ProbDist(no=25, mild=25, high=50),
                            F: ProbDist(no=97, mild=2, high=1)}),
    ('Körpertemperatur', ['Fieber'], {'no': ProbDist(unter38=95, über38=5),
                                       'mild': ProbDist(unter38=50, über38=50),
                                       'high': ProbDist(unter38=5, über38=95)}),
    ('Kopfweh', ['Grippe'], {T: 0.5, F: 0.03}) ])

# 4.3	Netzte mithilfe von Excel-Datei definieren

In [ ]:
einbruch_netz = get_net_from_xls("data/burglary.xls")

In [ ]:
cov_net = get_net_from_xls("data/COVID19-Netz.xls")

# 4.4	Netze inspizieren

In [ ]:
einbruch_netz.variables

In [ ]:
cov_net.variables

In [ ]:
einbruch_netz.get_node('Alarm')

In [ ]:
grippe_netz.variable_values('Fieber')
# or
grippe_netz.get_node('Fieber').variable_values

In [ ]:
einbruch_netz.get_node('Alarm').as_table()

# 4.5	Inferenz

In [ ]:
enumeration_ask('Einbruch', {'MaryRuftAn':F, 'JohnRuftAn':T}, einbruch_netz)

In [ ]:
elimination_ask('Einbruch', {'MaryRuftAn':F, 'JohnRuftAn':T}, einbruch_netz)

In [ ]:
elimination_ask('Einbruch', {'MaryRuftAn':F, 'JohnRuftAn':T}, einbruch_netz)[F]

In [ ]:
elimination_ask('Fieber', {'Grippe':T}, grippe_netz)["high"]

In [ ]:
elimination_ask('Eventual Covid status', {"age":"65-80",
                          "Underlying medical condition":"High",
                          "Intractions with other people":"Multiple external",
                          "Contact with symptomatic person":T,
                          "Frontline healthcare worker":F,
                          "Time since infection ifinfected":"<= 5 days",
                          "Test type": "PCR Nasal",
                          "Tested result": "Positive",
                          "Previous symptoms":"No",
                          "Previous validated test":"No"}, cov_net)

In [ ]:
elimination_ask('Eventual Covid status', {"age":"50-65",
                          "Underlying medical condition":"Low",
                          "Intractions with other people":"Multiple external",
                          "Contact with symptomatic person":T,
                          "Frontline healthcare worker":F,
                          "Time since infection ifinfected":"<= 5 days",
                          "Test type": "CT-scan",
                          "Tested result": "Positive",
                          "Previous symptoms":"No",
                          "Previous validated test":"No"}, cov_net)

# 4.6	Arbeiten mit Faktoren

In [ ]:
factor = Factor.make_factor('Alarm', {'JohnRuftAn':T,'MaryRuftAn':F}, einbruch_netz)

In [ ]:
factor.as_table()

In [ ]:
factor2 = Factor.make_factor('JohnRuftAn', {'JohnRuftAn':T,'MaryRuftAn':F}, einbruch_netz)
factor2.as_table()

In [ ]:
result = factor.pointwise_product(factor2, einbruch_netz)
result.as_table()

In [ ]:
result = result.sum_out('Alarm', einbruch_netz)
result.as_table()

# 4.7	Betreiben der Webapplikation
Oder run_server.py ausführen

In [ ]:
from dash_app.app import app

In [ ]:
app.run_server()

# 4.8	Mögliche Fehler

In [ ]:
einbruch_netz = BayesNet([
    ('Einbruch', '', 0.001),
    ('Erdbeben', '', 0.002),
    ('Alarm', 'Einbruch', # Elternteil vergessen
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnRuftAn', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryRuftAn', 'Alarm', {T: 0.70, F: 0.01}) ])

In [ ]:
grippe_netz = BayesNet([
    ('Geimpft', [], 0.60),
    ('Grippe', ['Geimpft'], {T: 0.002, F: 0.02}),
    ('Fieber', ['Grippe'], {T: ProbDist(kein=25, mild=25, hoch=50), # andere Werte der Verteilung
                            F: ProbDist(no=97, mild=2, high=1)}), # andere Werte der Verteilung
    ('Körpertemperatur', ['Fieber'], {'no': ProbDist(unter38=95, über38=5),
                                       'mild': ProbDist(unter38=50, über38=50),
                                       'high': ProbDist(unter38=5, über38=95)}),
    ('Kopfweh', ['Grippe'], {T: 0.5, F: 0.03}) ])

In [ ]:
grippe_netz = BayesNet([
    ('Geimpft', [], 0.60),
    ('Grippe', ['Geimpft'], {T: 0.002, F: 0.02}),
    ('Fieber', ['Grippe'], {T: ProbDist(no=25, mild=25, high=50),
                            F: ProbDist(no=97, mild=2, high=1)}),
    ('Körpertemperatur', ['Fieber'], {'no': ProbDist(unter38=95, über38=5),
                                       'mild': ProbDist(unter38=50, über38=50),
                                       'hoch': ProbDist(unter38=5, über38=95)}), # diesen Elternwert gibt es nicht
    ('Kopfweh', ['Grippe'], {T: 0.5, F: 0.03}) ])

In [ ]:
einbruch_netz = BayesNet([
    ('Einbruch', '', 0.001),
    ('Erdbeben', '', 0.002),
    ('Alarm', 'Einbruch Erdbeben',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29}), # (F, F) vergessen
    ('JohnRuftAn', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryRuftAn', 'Alarm', {T: 0.70, F: 0.01}) ])